In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import math
import datetime

Here we integrate the weather data and the bird data!

In [2]:
weather = pd.read_csv("Data/weather_seven.txt")
weather = weather[weather['YEAR'] != 2024]
weather = weather.dropna()
weather.loc[:,'PRCP'] = weather['PRCP']/ weather['PRCP_CNT']
weather.loc[:,'SNOW'] = weather['SNOW']/ weather['SNOW_CNT']
weather.loc[:,'TMAX'] = weather['TMAX']/ weather['TMAX_CNT']
weather.loc[:,'TMIN'] = weather['TMIN']/ weather['TMIN_CNT']

weather = weather.drop(['PRCP_CNT', 'SNOW_CNT', 'TMAX_CNT', 'TMIN_CNT'], axis = 1)
print(weather)

         YEAR  MONTH  DAY  LATITUDE  LONGITUDE        PRCP  SNOW        TMAX  \
0        2017      1    1      28.0      -97.0   40.421053   0.0  229.222222   
1        2017      1    1      28.0      -96.0  131.500000   0.0  232.200000   
2        2017      1    1      28.0      -83.0    0.000000   0.0  270.166667   
3        2017      1    1      28.0      -82.0    0.000000   0.0  257.625000   
4        2017      1    1      28.0      -81.0    0.000000   0.0  245.714286   
...       ...    ...  ...       ...        ...         ...   ...         ...   
1175823  2023     12   31      48.0      -79.0    0.000000   0.0  -71.000000   
1175824  2023     12   31      48.0      -78.0    0.666667   2.0  -71.500000   
1175825  2023     12   31      48.0      -73.0    0.000000   0.0  -66.500000   
1175826  2023     12   31      48.0      -72.0    1.500000   2.0  -71.800000   
1175827  2023     12   31      48.0      -71.0    0.000000  10.0  -59.500000   

               TMIN  
0        144.2222

In [3]:
species = {0 : 'amegfi', 1 : 'tuftit', 2 : 'daejun', 3 : 'mallar'}
agg = [[],[],[],[]]



for i in range(4):
    for j in range(7):
        agg[i].append(pd.read_csv('Data/{}/{}_ebd_{}_Feb-2024_aggregate.csv'.format(species[i], str(2017+j), species[i])))
    agg[i] = pd.concat(agg[i])
    agg[i]['SPECIES'] = i
    agg[i] = pd.merge(agg[i], 
                      weather,
                      how="left",
                      on=['YEAR', 'MONTH', 'DAY', 'LATITUDE', 'LONGITUDE']).set_index(
        ['YEAR', 'MONTH', 'DAY', 'LATITUDE', 'LONGITUDE'])
    print(agg[i])

    
    
data = pd.concat(agg)
data = data.reset_index(level=[0,1,2,3,4])

print(data)


                                   OBSERVATION COUNT  IBA CODE  BCR CODE  \
YEAR MONTH DAY LATITUDE LONGITUDE                                          
2017 1     1   28.0     -97.0                     11         0         1   
                        -83.0                     49         1        12   
                        -82.0                     66         2        18   
                        -81.0                     21         1         4   
               29.0     -97.0                     20         0         1   
...                                              ...       ...       ...   
2023 12    31  48.0     -73.0                      1         0         1   
                        -72.0                     14         0         2   
                        -71.0                     36         0         2   
                        -70.0                      1         0         1   
                        -69.0                      4         0         2   

           

                                   OBSERVATION COUNT  IBA CODE  BCR CODE  \
YEAR MONTH DAY LATITUDE LONGITUDE                                          
2017 1     1   28.0     -83.0                     41         0         4   
                        -82.0                    126         1        18   
                        -81.0                      3         1         1   
               29.0     -96.0                    258         2        23   
                        -95.0                      8         0         2   
...                                              ...       ...       ...   
2023 12    31  48.0     -90.0                    140         0         1   
                        -72.0                     30         0         1   
                        -71.0                     79         0         1   
                        -70.0                     28         2         3   
                        -69.0                     20         0         2   

           

Here we average all the information by day

In [10]:
data1 = data.dropna()
data1.loc[:,'OBSERVATION COUNT'] = data1['OBSERVATION COUNT']/ data1['COUNT']
data1.loc[:,'IBA CODE'] = data1['IBA CODE']/ data1['COUNT']
data1.loc[:,'BCR CODE'] = data1['BCR CODE']/ data1['COUNT']
data1.loc[:,'USFWS CODE'] = data1['USFWS CODE']/ data1['COUNT']
data1.loc[:,'DURATION MINUTES'] = data1['DURATION MINUTES']/ data1['COUNT']
data1 = data1.drop(['COUNT'], axis = 1)


print(data1)
print(data1.info())

         YEAR  MONTH  DAY  LATITUDE  LONGITUDE  OBSERVATION COUNT  IBA CODE  \
0        2017      1    1      28.0      -97.0          11.000000  0.000000   
1        2017      1    1      28.0      -83.0           4.083333  0.083333   
2        2017      1    1      28.0      -82.0           3.666667  0.111111   
3        2017      1    1      28.0      -81.0           4.200000  0.200000   
4        2017      1    1      29.0      -97.0          20.000000  0.000000   
...       ...    ...  ...       ...        ...                ...       ...   
2486271  2023     12   31      46.0      -73.0           8.000000  0.000000   
2486272  2023     12   31      46.0      -72.0          55.000000  0.187500   
2486273  2023     12   31      48.0      -90.0         140.000000  0.000000   
2486274  2023     12   31      48.0      -72.0          30.000000  0.000000   
2486275  2023     12   31      48.0      -71.0          79.000000  0.000000   

         BCR CODE  USFWS CODE  DURATION MINUTES  SP

In [11]:
data1["dsreal"] = data1.apply(lambda x: datetime.date(int(x['YEAR']),int(x['MONTH']),int(x['DAY'])),axis=1)

In [12]:
 def is_in(sth, iterable):
    for item in iterable:
        if sth == item:
            return True
    return False

## Calculate n where (year, month, and day) is in the n-th day since January 1st 2017

def ds_since_2017(year, month, day):
    if (year < 2017) or (month < 1) or (day < 1):
        print(year)
        print(month)
        print(day)
        print('Error: wrong date')
        return None
    leap_year = False
    if year%4 == 0:
        leap_year = True
    if month == 1:
        return day + 365 * (year - 2017) + (year >= 2020)
    else:
        if is_in(month, [4,6,9,11]):
            return ds_since_2017(year, month-1, day+30)
        elif month == 2:
            return ds_since_2017(year, month-1, day+28+leap_year)
        else:
            return ds_since_2017(year, month-1, day+31)

    
data1['ds'] = data1.apply(lambda x: ds_since_2017(x['YEAR'], x['MONTH'],x['DAY']), axis=1)

In [13]:
print(data1)

         YEAR  MONTH  DAY  LATITUDE  LONGITUDE  OBSERVATION COUNT  IBA CODE  \
0        2017      1    1      28.0      -97.0          11.000000  0.000000   
1        2017      1    1      28.0      -83.0           4.083333  0.083333   
2        2017      1    1      28.0      -82.0           3.666667  0.111111   
3        2017      1    1      28.0      -81.0           4.200000  0.200000   
4        2017      1    1      29.0      -97.0          20.000000  0.000000   
...       ...    ...  ...       ...        ...                ...       ...   
2486271  2023     12   31      46.0      -73.0           8.000000  0.000000   
2486272  2023     12   31      46.0      -72.0          55.000000  0.187500   
2486273  2023     12   31      48.0      -90.0         140.000000  0.000000   
2486274  2023     12   31      48.0      -72.0          30.000000  0.000000   
2486275  2023     12   31      48.0      -71.0          79.000000  0.000000   

         BCR CODE  USFWS CODE  DURATION MINUTES  SP

In [14]:
data1.to_csv("Data/AllSpecYear.csv",
          index=False)